In [4]:
from keras import layers
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import gym
import time

In [5]:
env = gym.make('SkiingDeterministic-v4')
action_size = env.action_space.n

In [6]:
def get_frame_reward(I, prev):
    I = I[:, :, 1]
    I = I[74:75, 8:152]  # Jugador 92, bandera roja 50, bandera azul 72
    if 72 not in I and 50 not in I:
        return 0
    if 72 in I:
        flags = np.where(I == 72)
    else:
        flags = np.where(I == 50)

    player = np.where(I == 92)[1]

    if len(player) == 0:
        return 1

    player = player.mean()

    if len(flags[1]) == 2:
        if player >= flags[1][0] and player <= flags[1][1]:
            return 1
        else:
            return -1
    else:
        return prev

# reward discount used by Karpathy (cf. https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5)
def discount_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    r = np.array(r)
    discounted_r = np.zeros_like(r)
    running_add = 0
    # we go from last reward to first one so we don't have to do exponentiations
    for t in reversed(range(0, r.size)):
        if r[t] != 0:
            # if the game ended (in Pong), reset the reward sum
            running_add = 0
        # the point here is to use Horner's method to compute those rewards efficiently
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    

    discounted_r -= np.mean(discounted_r)  # normalizing the result
    discounted_r /= np.std(discounted_r)  # idem

    return discounted_r

#other code
def get_pos_player(observe):
  ids = np.where(np.sum(observe == [214, 92, 92], -1) == 3)
  return ids[0].mean(), ids[1].mean()

def get_pos_flags(observe):
  if np.any(np.sum(observe == [184, 50, 50], -1) == 3):
    ids = np.where(np.sum(observe == [184, 50, 50], -1) == 3)
    return ids[0].mean(), ids[1].mean()
  else:
    base = 0
    ids = np.where(np.sum(observe[base:-60] == [66, 72, 200], -1) == 3)
    return ids[0].mean() + base, ids[1].mean()

def get_speed(observe, observe_old):
  min_val = np.inf
  min_idx = 0
  for k in range(0, 7):
    val = np.sum(np.abs(observe[54:-52,8:152] - observe_old[54+k:-52+k,8:152]))
    if min_val > val:
      min_idx = k
      min_val = val
  return min_idx


In [7]:
class Skier:
    def __init__(self, gamma=0.95, epsilon=0.05, e_min=0.05, e_decay=0.99, ideal_flag_interval=25):
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = e_min
        self.epsilon_decay = e_decay

        self.episode = 0

        self.ideal_flag_interval = ideal_flag_interval

        self.autosave = None

        self.model = self._make_model()

        self.reset()

    def _make_model(self):
        model = Sequential()

        model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(146, 144, 2)))
        model.add(layers.Conv2D(8, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(16, (3, 3), activation='relu'))
        model.add(layers.Conv2D(16, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(units=256,activation='relu'))
        model.add(layers.Dense(units=3,activation='softmax'))
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model

    def preprocessFrame(self, I):
        """ 
        Outputs a 72x72 image where background is black
        and important game elements are white.
        Output is [0,1]
        """
        I = I[57:203, 8:152, 1]
        return I/255

    def decay(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def reset(self):
        self.last_reward = 0
        self.rewards = []
        self.train_x = []
        self.train_y = []
        self.lr_counter = 0
        self.missing_flags = 20
        self.last_frame = np.zeros((146, 144))

    def action(self, frame, training=False):
        frame = self.preprocessFrame(frame)
        data = np.zeros((146, 144, 2))
        data[:,:,0] = self.last_frame
        data[:,:,0] = frame
        self.last_frame = frame
        probs = self.model.predict(np.expand_dims(data, 0))
        #y = np.random.choice([0, 1, 2], p=probs[0])
        y = np.argmax(probs[0])
        
        if float('nan') in probs[0]:
            print("NANANANANANANANANANANANANANANANANANA", probs[0])
            exit()

        # Append flattened frame to x_train
        self.train_x.append(data)
        # Append selected action to y_train
        self.train_y.append(to_categorical(y, num_classes=3))
        return y

    def register_frame(self, frame, rew = 0):
        frame_reward = get_frame_reward(frame, self.last_reward)
        reward = rew
        self.lr_counter += 1
        if frame_reward == 0 and self.last_reward != 0:
            reward += self.last_reward + 2
            reward -= 0.25 * np.tanh((0.05 * (self.lr_counter - self.ideal_flag_interval)))
            self.lr_counter = 0
            self.missing_flags -= 1
        self.last_reward = frame_reward

        self.rewards.append(reward)

    def done(self):
        self.rewards[-1] -= self.missing_flags * 5

    def train(self, verbose=0):
        if self.autosave is not None and self.episode % self.autosave == 0:
            self.save("last_lf.h5")
            print("Saved!")
        
        #print("missed:", self.missing_flags, "flags")
        #self.rewards[-1] -= self.missing_flags * 5
        sample_weights = discount_rewards(self.rewards, self.gamma)
        self.model.fit(
            x=np.array(self.train_x),
            y=np.vstack(self.train_y),
            verbose=verbose,
            sample_weight=sample_weights
        )
        self.episode += 1
        self.decay()

    def total_reward(self):
        return np.array(self.rewards).sum()

    def set_autosave(self, interval):
        self.autosave = interval

    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)


In [8]:
agent = Skier(gamma=0.99, e_decay=0.995)
agent.model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 144, 142, 8)       152       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 142, 140, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 71, 70, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 68, 16)        1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 67, 66, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 16)        0         
_________________________________________________________________
flat

In [14]:

agent.load("last_lf_best.h5")

agent.set_autosave(10)
observation = env.reset()
agent.reset()

observation_old = observation

#other code

cnt = 0
r_a, c_a = get_pos_player(observation)
r_f, c_f = get_pos_flags(observation)
r_a_old, c_a_old = r_a, c_a
#------------


while True:
    env.render()
    #action = agent.action(observation, training=True)
    

    #other code
    # TEACHER
    v_f = np.arctan2(r_f - r_a, c_f - c_a) # direction from player to target
    spd = get_speed(observation, observation_old)
    v_a = np.arctan2(spd, c_a - c_a_old) # speed vector of the player
    r_a_old, c_a_old = r_a, c_a
    
    
    
    frame = agent.preprocessFrame(observation)
    data = np.zeros((146, 144, 2))
    data[:,:,0] = agent.last_frame
    data[:,:,1] = frame
    agent.last_frame = frame
        
    
    observation_old = observation
    if np.random.rand() <= agent.epsilon:
        if spd == 0 and (c_a - c_a_old) == 0:
            cnt += 1
            act_t = np.random.choice(3, 1)[0]
        else:
            cnt = 0
            if v_f - v_a < -0.1:
                act_t = 1
            elif v_f - v_a > 0.1:
                act_t = 2
            else:
                act_t = 0

    else:
        if spd == 0 and (c_a - c_a_old) == 0:
            cnt += 1
            act_t = np.random.choice(3, 1)[0]
        else:
            cnt = 0
            probs = agent.model.predict(np.expand_dims(data, 0))
            act_t = np.argmax(probs[0])
            if float('nan') == probs[0][0]:
                print("NANANANANANANANANANANANANANANANANANA", probs[0])
                exit()


    
    # Append flattened frame to x_train
    agent.train_x.append(data)
    # Append selected action to y_train
    agent.train_y.append(to_categorical(act_t, num_classes=3))
    
    observation, reward, done, info = env.step(act_t)
    r_a, c_a = get_pos_player(observation)
    r_f, c_f = get_pos_flags(observation)
    #------------

    
    
    if cnt > 30:
        done = True
        agent.register_frame(observation,-100.0)
        cnt = 0
    else:
        agent.register_frame(observation)
    
    if done:
        #agent.done()
        total_reward = agent.total_reward()

        if agent.episode % 1 == 0:
            print('\n# - = - = - = - #')
            print(
                f"Ep: {agent.episode:4}\nTotal reward: {total_reward:.3f}\nEpsilon: {agent.epsilon:.4f}")
        agent.train(verbose=1)
        agent.reset()

        observation = env.reset()


# - = - = - = - #
Ep:    0
Total reward: 24.162
Epsilon: 0.0500
Saved!
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
782/782 [==============================] - 3s 4ms/step - loss: -0.8364 - acc: 0.8721

# - = - = - = - #
Ep:    1
Total reward: 40.514
Epsilon: 0.0500
Epoch 1/1
667/667 [==============================] - 2s 3ms/step - loss: 0.9114 - acc: 0.8906

# - = - = - = - #
Ep:    2
Total reward: 29.323
Epsilon: 0.0500
Epoch 1/1
944/944 [==============================] - 2s 2ms/step - loss: -0.3230 - acc: 0.8390

# - = - = - = - #
Ep:    3
Total reward: 46.369
Epsilon: 0.0500
Epoch 1/1
763/763 [==============================] - 2s 2ms/step - loss: -0.1397 - acc: 0.8650

# - = - = - = - #
Ep:    4
Total reward: 30.035
Epsilon: 0.0500
Epoch 1/1
871/871 [==============================] - 2s 2ms/step - loss: -0.7441 - acc: 0.8370

# - = - = - = - #
Ep:    5
Total reward: 34.004
Epsilon: 0.0500
Epoch 1/1
779/779 [==============================] - 2s 2ms/step - loss: -0.1941 

KeyboardInterrupt: 

In [15]:
agent.save("last_lf_best.h5")
env.close()

In [18]:
#agent.load("last_lf_best.h5")

agent.set_autosave(10)
observation = env.reset()
agent.reset()

observation_old = observation

#other code

cnt = 0
r_a, c_a = get_pos_player(observation)
r_f, c_f = get_pos_flags(observation)
r_a_old, c_a_old = r_a, c_a
#------------


while True:
    env.render()
    #action = agent.action(observation, training=True)
    

    #other code
    # TEACHER
    v_f = np.arctan2(r_f - r_a, c_f - c_a) # direction from player to target
    spd = get_speed(observation, observation_old)
    v_a = np.arctan2(spd, c_a - c_a_old) # speed vector of the player
    r_a_old, c_a_old = r_a, c_a
    
    
    
    frame = agent.preprocessFrame(observation)
    data = np.zeros((146, 144, 2))
    data[:,:,0] = agent.last_frame
    data[:,:,0] = frame
    agent.last_frame = frame
        
    
    observation_old = observation

    if spd == 0 and (c_a - c_a_old) == 0:
        cnt += 1
        act_t = np.random.choice(3, 1)[0]
    else:
        cnt = 0
        probs = agent.model.predict(np.expand_dims(data, 0))
        act_t = np.argmax(probs[0])

        if float('nan') in probs[0]:
            print("NANANANANANANANANANANANANANANANANANA", probs[0])
            exit()

    observation, reward, done, info = env.step(act_t)

    if done:
        break
env.close()